## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [1]:
class Matter:
    """Wrapper for the properties of the matter sold by a specific farm:
    - purchase: the purchase price
    - storage: the storage cost
    - dry_perc: the percentage of dry matter
    """

    def __init__(
            self,
            purchase: float,
            storage: float,
            dry_perc: int) -> None:
        self.purchase = purchase
        self.storage = storage
        self.dry_perc = dry_perc


In [2]:
import mip

m = mip.Model()

max_production = 1950000    # kWh/quarter
energy_per_ton = 421.6      # kWh/ton
income_per_energy = .28     # $/kWh
max_storage = 500           # tons

farms = 5
quarters = 8

productions = [
    [700,   1500,   700,    0,  0,       700,    1500,   0],
    [1350,  0,      450,    0,  1350,    0,      450,    0],
    [0,     1500,   1500,   0,  0,       1500,   1500,   0],
    [820,   1560,   820,    0,  820,     1560,   820,    0],
    [0,     680,    1080,   0,  0,       680,    1080,   0]
]

matters = [
    Matter(.20, .002, 15),
    Matter(.18, .012, 28),
    Matter(.19, .007, 35),
    Matter(.21, .011, 37),
    Matter(.23, .015, 42)
]


In [3]:
# Variables

# Amount of matter stored into the silo per quarter.
# Rows are quarters, columns are farms.
silo = [[m.add_var() for _ in range(farms)] for _ in range(quarters)]
# First row is full of zeros and represents the state of the silo before
# the first quarter.
silo.insert(0, [0. for _ in range(farms)])

# Amount purchased for each farm and for each quarter.
purchased = [[m.add_var() for _ in range(quarters)] for _ in range(farms)]

# Amount used for each farm and for each quarter.
used = [[m.add_var() for _ in range(quarters)] for _ in range(farms)]


In [4]:
# Constraints

for j in range(quarters):
    # The maximum space inside the silo is 500 tons.
    m.add_constr(mip.xsum(silo[j + 1]) <= max_storage)
    # The used amount cannot produce more than the maximum legal production value.
    m.add_constr(mip.xsum(f[j] for f in used) * energy_per_ton <= max_production)
    # The dry matter percentage of the mix must be between 20% and 40%.
    m.add_constr(
        mip.xsum(used[f][j] * matters[f].dry_perc for f in range(farms)) >=
        20 * mip.xsum(used[f][j] for f in range(farms)))
    m.add_constr(
        mip.xsum(used[f][j] * matters[f].dry_perc for f in range(farms)) <=
        40 * mip.xsum(used[f][j] for f in range(farms)))
    for i in range(farms):
        # The purchased amount cannot go over the maximum farm production.
        m.add_constr(purchased[i][j] <= productions[i][j])
        # Purchased and not used matter must be stored into the silo.
        m.add_constr(silo[j + 1][i] == silo[j][i] + purchased[i][j] - used[i][j])


In [5]:
# Objective function

# We want to maximize the revenue.
m.objective = mip.maximize(
    # Incomes from production
    mip.xsum(used[i][j] for i in range(farms) for j in range(quarters)) * energy_per_ton * income_per_energy -
    # Purchase of matter
    mip.xsum(purchased[i][j] * matters[i].purchase for i in range(farms) for j in range(quarters)) -
    # Storage cost
    mip.xsum(silo[i + 1][j] * matters[j].storage for i in range(quarters) for j in range(farms))
)
# m.verbose = 0     # To reduce output from mip solver.
m.optimize()


Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming problem using Dual Simplex

Coin0506I Presolve 54 (-58) rows, 84 (-36) columns and 269 (-86) elements
Clp0000I Optimal - objective value 2861373.9
Coin0511I After Postsolve, objective 2861373.9, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 2861373.925 - 35 iterations time 0.002, Presolve 0.00


<OptimizationStatus.OPTIMAL: 0>

In [6]:
# Results of optimization

# Maximum revenue
print(f'The maximum revenue is $ {m.objective_value}.')

# Revenue per quarter
revenue = [
    sum(used[i][j].x for i in range(farms)) * energy_per_ton * income_per_energy -
    sum(purchased[i][j].x * matters[i].purchase for i in range(farms)) -
    sum(silo[j + 1][i].x * matters[i].storage for i in range(farms))
    for j in range(quarters)
]

# Energy per quarter
energy_per_quarter = [
    sum(f[i].x for f in used) * energy_per_ton
    for i in range(quarters)
]

# Percentage of dry matter per quarter
percentages_result = [
    sum(used[f][i].x * matters[f].dry_perc for f in range(farms)) /
    sum(used[f][i].x for f in range(farms))
    for i in range(quarters)
]

# Purchased amounts of matter per quarter and per farm
purchased_result = [
    [purchased[j][i].x for i in range(quarters)]
    for j in range(farms)
]

# Used amounts of matter per quarter and per farm
used_result = [
    [used[j][i].x for i in range(quarters)]
    for j in range(farms)
]

# Stored amounts of matter per quarter and per farm
silo_result = [
    [silo[i + 1][j].x for i in range(quarters)]
    for j in range(farms)
]


The maximum revenue is $ 2861373.9254127136.
